<a href="https://colab.research.google.com/github/koki0702/zerobook3/blob/master/notebook/ja/step05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 前ステップまでに実装したコード

In [1]:
import numpy as np


class Variable:
    def __init__(self, data):
        self.data = data


class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        return x ** 2


class Exp(Function):
    def forward(self, x):
        return np.exp(x)


def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

***

# ステップ5 バックプロパゲーションの理論

私たちは数値微分によって微分を求めることができました。しかし、数値微分には計算コストと精度の点で問題がありました。バックプロパゲーションは、その2つの問題を解決します！ つまりは、微分を効率良く求めることができる上に、より誤差の小さい値を得ることができるのです。本ステップは、バックプロパゲーションの実装は行わずに理論の説明だけに留めます。そして、次のステップからバックプロパゲーションの実装を行っていきます。

## 5.1 チェインルール

バックプロパゲーションを理解する上でキーとなるのが**チェインルール（連鎖律）**です。チェイン（chain）とは「鎖」を意味し、複数の関数が連結して使われる様子を表します。チェインルールは、その連結した複数の関数（合成関数）の微分が、それを構成する各関数の微分の積へと“分解”できることを表します。

チェインルールについて具体例を出して説明しましょう。たとえばここに、$y=F(x)$という関数があるとします。そして、この関数$F$が、$a=A(x)$、$b=B(a)$、$y=C(b)$の3つの関数によって構成されているとしましょう。ちなみに、この関数は計算グラフで書くと、**図5-1**のようになります。

<br>![variable_function](images/1-7.png)
<br>**図5-1** 合成関数の例<br><br>


このとき、$x$に関する$y$の微分は**図5-1**で表すことができます。

$$
\frac{dy}{dx} = \frac{dy}{db} \frac{db}{da} \frac{da}{dx} \tag{5.1}
$$

式(5.1)が示すように、$x$に関する$y$の微分は、各関数の微分の積によって表されます。つまり、合成関数の微分は、各関数の局所的な微分へと分解できるということです。これがチェインルールです。また、式(5.1)で表されるチェインルールは、次のように、$\frac{dy}{dy}$を明示的に含めて書くこともできます。

$$
\frac{dy}{dx} = \frac{dy}{dy} \frac{dy}{db} \frac{db}{da} \frac{da}{dx} \tag{5.2}
$$

$\frac{dy}{dy}$は「自分自身」に関しての微分であり、その値は常に1になります。そのため、$\frac{dy}{dy}$のような「自分自身」に関する微分の積は省略するのが普通ですが、ここではバックプロパゲーションの実装を見越して含めることにします。

<div class="alert alert-info">
<b>【ノート】</b>$\frac{dy}{dy}$は、$y$の$y$に関する微分です。このとき、$y$がある微小値だけ変化すると、自分自身である$y$も同じ量だけ変化します。そのため、その変化の割合は、どのような関数の場合でも常に$1$になります。
</div>

## 5.2 バックプロパゲーションの導出

それでは、式(5.2)をじっくりと見ていきましょう。式(5.2)は、合成関数の微分が各関数の微分の積へと分解できることを意味します。しかし、それを「どの順番で掛け算するか」までは明言していません。もちろん、その点は自由に決めることができます。そこで、式(5.3)のように、出力から入力方向へと順に計算していくことを考えます（[注釈1](#note1)）。

$$
\frac{dy}{dx} =   \biggl( \biggl(\frac{dy}{dy} \frac{dy}{db} \biggl) \frac{db}{da}\biggl) \frac{da}{dx} \tag{5.3}
$$

式(5.3)のように、出力から入力方向へ――つまり、通常の計算とは逆方向へ――微分の計算を行います。このとき、式(5.3)の計算の流れは**図5-2**のようになります。

<br>![variable_function](images/1-8.png)
<br>**図5-2** 出力側の微分から順に計算する流れ<br><br>

**図5-2**のとおり、出力$y$から入力$x$の方向へと掛け算をしながら順に微分を計算していきます。そうすることで、最終的に$\frac{dy}{dx}$が求まります。この計算を「計算グラフ」で表すと、**図5-3**のようになります。

<br>![variable_function](images/1-9.png)
<br>**図5-3** $\frac{dy}{dx}$を求める計算グラフ<br><br>

**図5-3**の計算グラフをよく観察してみましょう。まずは、$\frac{dy}{dy} (=1)$からスタートし、$\frac{dy}{db}$との積を計算します。ここで、$\frac{dy}{db}$は、$y=C(b)$という関数の微分です。そのため、関数$C$の導関数を$C'$で表せば$\frac{dy}{db}=C'(b)$と書けます。同様に、$\frac{db}{da}=B'(a)$、$\frac{da}{dx}=A'(x)$となります。以上の点を考慮すると、**図5-3**の計算グラフは、次のように簡略化して書くことができます。

<br>![variable_function](images/1-10.png)
<br>**図5-4** 簡略化した逆伝播の計算グラフ（$A'(x)$の乗算は「$A'(x)$」というノードで簡略化して表す<br><br>

**図5-4**のとおり、導関数との積を1つの関数ノードで表すことにします。これで、微分の流れが明確になります。**図5-4**を見ると、右から左へと「$y$の各変数に関する微分」が伝播することが分かります。これが逆伝播です。ここで重要な点は、伝播するデータは、すべてが「$y$の微分」だということです。具体的に書くと、$\frac{dy}{dy}$、$\frac{dy}{db}$、$\frac{dy}{da}$、$\frac{dy}{dx}$のように、すべて「$y$の〇〇に関する微分」が伝播していることが分かります。

<br><div class="alert alert-info">
<b>【ノート】</b>式(5.3)のように出力から入力方向へと計算の順番を指定した理由は、$y$の微分を伝播するためです。言い換えると、$y$を「重要人物」として扱うためです。もし仮に入力から出力方向へと順に計算を行ったとしたら、入力である$x$が「重要人物」になります。その場合、伝播する微分は、$\frac{dx}{dx}$ → $\frac{da}{dx}$ → $\frac{db}{dx}$ → $\frac{dy}{dx}$となり、$x$に関する微分が伝播することになります。
</div><br>

機械学習の多くの問題は、大量のパラメータを入力として、「損失関数」を最終出力とする形で定式化できます。この損失関数の出力は、（多くの場合）1つのスカラ値であり、これが「重要人物」になります。つまり、損失関数の各パラメータに関する微分を求める必要があるのです。そのような場合、微分を出力から入力方向へと伝播すれば、一度の伝播だけですべてのパラメータに関する微分を求めることができます。この計算効率の良さから、微分を逆方向に伝播する方式が用いられます。


<hr />

<div class="alert alert-info">
**注釈 1**

入力から出力方向へと順に計算するようなカッコの付け方も考えられます。それに基づく手法は「フォワードモードの自動微分」と呼ばれます。フォワードモードの自動微分については、本書の「コラム：自動微分」で説明します。

[▲上へ戻る](#ref_note1)
</div>